# CarND-Term1 Project 5: Vehicle Detection (by Atif)

Detect Vehicles on the road. 

##### Import all libraries used in the model

In [ ]:
import glob
import cv2
import numpy as np
from matplotlib import pyplot as plt
from moviepy.video.io.VideoFileClip import VideoFileClip
from scipy import misc
from skimage.feature import hog 
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import pickle 
import os

from pathlib import Path

##### Extraction of features from a images. 

In [2]:
def extract_features(img, cspace='RGB', spatial_size=(32, 32),
                        hist_bins=32, color_range=(0, 256)) : 

        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(img, cv2.COLOR_RGB2YUV)
        else: feature_image = np.copy(img)

        # Apply bin_spatial() to get spatial color features
        spatial_features = cv2.resize(feature_image,spatial_size).ravel()

        # Apply color_hist() also with a color space option now
        channel1_hist = np.histogram(feature_image[:,:,0], bins=hist_bins, range=color_range)
        channel2_hist = np.histogram(feature_image[:,:,1], bins=hist_bins, range=color_range)
        channel3_hist = np.histogram(feature_image[:,:,2], bins=hist_bins, range=color_range)
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
        
        # Compute the HOG features for input image 
        hog_features = []
        for channel in range(feature_image.shape[2]):
            hog_features.append(hog(feature_image[:, :, channel], orientations=9, pixels_per_cell=(8,8),
                           cells_per_block=(2,2), transform_sqrt=True, 
                           visualise=False, feature_vector=True))
        hog_features = np.ravel(hog_features)
        
        # Append the new feature vector to the features list and Return list of feature vectors
        features = np.concatenate((spatial_features, hist_features, hog_features))
        return features

def extract_features_2(img, spatial_size=(32, 32),
                        hist_bins=32, color_range=(0, 256)) : 
        rgb_features = extract_features(img, 'RGB', spatial_size, hist_bins, color_range)
        hsv_features = extract_features(img, 'HSV', spatial_size, hist_bins, color_range)
        return np.concatenate([rgb_features, hsv_features])
        

In [3]:
feature_set = []
output=[]

images = glob.glob('vehicles/*/*.png')
for imname in images:
        img = cv2.imread(imname)
        a = extract_features_2(img)
        feature_set.append(a)
output = [1 for i in range(len(feature_set))]
print(len(feature_set))

images = glob.glob('non-vehicles/*/*.png')
for imname in images :
        img = cv2.imread(imname)
        a = extract_features_2(img)
        feature_set.append(a)
output.extend([0 for i in range(len(feature_set)-len(output))])
print(len(feature_set))

outputY = np.array(output)
dataX = np.array(feature_set)
print(dataX.shape)
print(outputY.shape)


8792
17760
(17760, 16920)
(17760,)


##### Use test inputs and create a linear-SVM model for classifying 64x64x3 images as having Car or Not-Car 
Compute linear SVM for classifying images. 
Load existing model file if exists. 

In [ ]:
if os.path.isfile("./SVM.sav"):
    svm = pickle.load(open("SVM.sav",'rb'))

else:
    svm = SVC(kernel='linear')
    train_data,test_data ,train_labels ,test_labels  = train_test_split(dataX,outputY,test_size = 0.3,random_state=3)
    print(train_data.shape,test_labels.shape)
    svm.fit(train_data,train_labels)
    pred_labels = svm.predict(test_data)
    pickle.dump(svm, open("SVM.sav", 'wb'))
    print(confusion_matrix(test_labels,pred_labels))
    print(classification_report(test_labels,pred_labels))

###### Apply the predicted model to classify car/not-car label for a given window in frame 

In [ ]:
i = 10
def predict_car(image, window, trace=False):
    global svm
    global i
    #x_range = window[1][0] - window[0][0]
    #y_range = window[1][1] - window[0][1]
    if trace and i>0:
        plt.imshow(image)
        plt.show()
    cropped_image = image[window[0][1]:window[1][1],window[0][0]:window[1][0],:]
    if trace and i>0:
        plt.imshow(cropped_image)
        plt.show()
    resized_image =  cv2.resize(cropped_image,(64,64))
    if trace and i>0:
        plt.imshow(resized_image)
        plt.show()
    test_data     = extract_features_2(resized_image)
    pred_label = svm.predict(test_data) 
    if trace and i>0:
        print("predicted ", pred_label)
        i = i-1

    return pred_label
   

##### Slide fixed size windows across the image to detect presence of car 

In [ ]:
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched    
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step) 
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step) 
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    # Note: you could vectorize this step, but in practice
    # you'll be considering windows one by one with your
    # classifier, so looping makes sense
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list


##### Slide multiple sized windows within focus region of image (focus smaller for small window) to reduce search space, to detect presence of car. 

In [ ]:
def slide_variable_window(image,size_list):
    windows=[]
    for size in size_list:
          windows.extend((slide_window(image,  x_start_stop=[image.shape[1]//5-size//2, image.shape[1]*4//5+size//2], 
                                    y_start_stop=[image.shape[0]*2//3-size//2, image.shape[0]*9//10+size//2], 
                                    xy_window=(size, size), xy_overlap=(0.6, 0.6))))
    return windows

##### Augment input image with detected cars regions 
1. Slide-window across the Input image 
2. Identify presence of cars in each window
    * scale the window region, and extract features from it 
    * predict the label (car/no-car) for each window
3. Draw all car containing windows on input image

In [ ]:
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        #cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
        center = ((bbox[0][0]+bbox[1][0])//2, (bbox[0][1]+bbox[1][1])//2)
        cv2.circle(imcopy, center, (bbox[1][0]-bbox[0][0])//2, (np.random.rand()*255,np.random.rand()*255,np.random.rand()*255), thick)
    # Return the image copy with boxes drawn
    return imcopy
    

def augment (img): 
    global i
    window_list = slide_variable_window(base,[256,160,96,64])
    predicted_cars=[]
    for window in window_list:
      pred_label = predict_car(img, window)
      if pred_label == 1:
            predicted_cars.append(window)
            predict_car(img, window, True)

    augmented = draw_boxes(img, predicted_cars, color=(0, 0, 255), thick=6)         
    return augmented


In [ ]:
video_in = VideoFileClip('project_video'+'.mp4') #.subclip(30,51)
base = video_in.get_frame(47)

window_img = augment(base)
#print(predicted_cars)       
plt.imshow(window_img)
plt.show()

video_out = video_in.subclip(30,41).fl_image(augment)  # NOTE: this function expects color images!!
video_out.write_videofile('project_video'+'_augmented.mp4', audio=False)